In [ ]:
# Write as geotiff with rasterio
from pathlib import Path

import numpy as np
import rioxarray
import xarray as xr
from stopuhr import stopwatch

DATA_DIR = Path("./data/compression_tests")

In [ ]:
gen = np.random.default_rng(42)
x = np.arange(0, 10000)
y = np.arange(0, 10000)[::-1]
# 10m resolution
red = xr.DataArray(gen.random((10000, 10000)), dims=("y", "x"), coords={"x": x, "y": y})
green = xr.DataArray(gen.random((10000, 10000)), dims=("y", "x"), coords={"x": x, "y": y})
# 60m resolution
swir = xr.DataArray(gen.random((len(x[::6]), len(y[::6]))), dims=("y", "x"), coords={"x": x[::6], "y": y[::6]})
ds_multires = xr.Dataset({"red": red, "swir": swir.interp_like(red, method="nearest")})
display(ds_multires)
ds_singleres = xr.Dataset({"red": red, "green": green})
display(ds_singleres)

In [ ]:
with stopwatch("Saving multires to zarr"):
    ds_multires.to_zarr(DATA_DIR / "multires.zarr", mode="w", consolidated=False)
with stopwatch("Saving singleres to zarr"):
    ds_singleres.to_zarr(DATA_DIR / "singleres.zarr", mode="w", consolidated=False)
with stopwatch("Saving multires to netcdf"):
    ds_multires.to_netcdf(
        DATA_DIR / "multires.nc",
        engine="h5netcdf",
        mode="w",
        encoding={"red": {"zlib": True, "complevel": 5}, "swir": {"zlib": True, "complevel": 5}},
    )
with stopwatch("Saving singleres to netcdf"):
    ds_singleres.to_netcdf(
        DATA_DIR / "singleres.nc",
        engine="h5netcdf",
        mode="w",
        encoding={"red": {"zlib": True, "complevel": 5}, "green": {"zlib": True, "complevel": 5}},
    )

In [ ]:
with stopwatch("Saving multires to geotiff"):
    ds_multires.to_dataarray(dim="band").rio.to_raster(DATA_DIR / "multires.tif", compress="zstd")

with stopwatch("Saving singleres to geotiff"):
    ds_singleres.to_dataarray(dim="band").rio.to_raster(DATA_DIR / "singleres.tif", compress="zstd")

with stopwatch("Saving multires to cog"):
    ds_multires.to_dataarray(dim="band").rio.to_raster(DATA_DIR / "multires-cog.tif", compress="zstd", driver="COG")

with stopwatch("Saving singleres to cog"):
    ds_singleres.to_dataarray(dim="band").rio.to_raster(DATA_DIR / "singleres-cog.tif", compress="zstd", driver="COG")

In [ ]:
stopwatch.export().T

In [ ]:
with stopwatch("Reading multires from zarr"):
    ds_multires_zarr = xr.open_zarr(DATA_DIR / "multires.zarr", consolidated=False).load()

with stopwatch("Reading singleres from zarr"):
    ds_singleres_zarr = xr.open_zarr(DATA_DIR / "singleres.zarr", consolidated=False).load()

with stopwatch("Reading multires from netcdf"):
    ds_multires_netcdf = xr.open_dataset(DATA_DIR / "multires.nc", engine="h5netcdf").load()

with stopwatch("Reading singleres from netcdf"):
    ds_singleres_netcdf = xr.open_dataset(DATA_DIR / "singleres.nc", engine="h5netcdf").load()

In [ ]:
with stopwatch("Loading multires from geotiff"):
    ds_multires_geotiff = rioxarray.open_rasterio(DATA_DIR / "multires.tif").load().to_dataset(dim="band")

with stopwatch("Loading singleres from geotiff"):
    ds_singleres_geotiff = rioxarray.open_rasterio(DATA_DIR / "singleres.tif").load().to_dataset(dim="band")

with stopwatch("Loading multires from cog"):
    ds_multires_cog = (
        rioxarray.open_rasterio(DATA_DIR / "multires-cog.tif", driver="COG", overview_level=1)
        .load()
        .to_dataset(dim="band")
    )

with stopwatch("Loading singleres from cog"):
    ds_singleres_cog = (
        rioxarray.open_rasterio(DATA_DIR / "singleres-cog.tif", driver="COG", overview_level=1)
        .load()
        .to_dataset(dim="band")
    )

In [ ]:
ds_multires_geotiff = rioxarray.open_rasterio(DATA_DIR / "multires.tif").load().to_dataset(dim="band")
ds_multires_geotiff